In [1]:
import os

os.chdir("../../")
%pwd

'c:\\Users\\anfe1\\OneDrive\\Escritorio\\Instaleap\\Instamarket'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    data_path: Path
    clean_data_file: Path

In [3]:
from instamarket.constants import CONFIG_FILE_PATH
from instamarket.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self) -> None:
        config_file_path = CONFIG_FILE_PATH

        self.config = read_yaml(config_file_path)

        create_directories([self.config.artifacts_root])
    
    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        config = self.config.data_preprocessing

        create_directories([config.root_dir])

        data_preprocessing_config = DataPreprocessingConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            clean_data_file=config.clean_data_file
        )

        return data_preprocessing_config

In [4]:
import pandas as pd
import numpy as np

from instamarket.utils.common import save_object
from instamarket.logging import logger

class DataPreprocessing:
    def __init__(self, config:DataPreprocessingConfig) -> None:
        self.config = config

    def start(self) -> pd.DataFrame:
        logger.info("Read the dataset as dataframe")
        df = pd.read_csv(self.config.data_path)

        logger.info("Converting picking times to datetime DType")
        datetime_cols = ["actual_start_time_picking",
                        "actual_end_time_picking",
                        "optimal_start_time_picking",
                        "optimal_end_time_picking"]
        df[datetime_cols] = df[datetime_cols].apply(pd.to_datetime, format="mixed")
        
        df.drop(columns="job_id", axis=1, inplace=True)

        logger.info("Removing missing values")
        df.dropna(subset=['actual_start_time_picking', 'actual_end_time_picking'], inplace=True)

        logger.info("Adding columns for analysis") 
        df["actual_total_time"] = (df["actual_end_time_picking"] - df["actual_start_time_picking"]).dt.total_seconds() / 60.0
        df["optimal_total_time"] = (df["optimal_end_time_picking"] - df["optimal_start_time_picking"]).dt.total_seconds() / 60.0
        df["diff_actual_optimal_time"] = df["actual_total_time"] - df["optimal_total_time"]

        df["start_delay"] = (df["actual_start_time_picking"] - df["optimal_start_time_picking"]).dt.total_seconds() / 60.0
        df["end_delay"] = (df["actual_end_time_picking"] - df["optimal_end_time_picking"]).dt.total_seconds() / 60.0

        df["PTP"] = df["actual_total_time"]/df["optimal_total_time"]

        logger.info("Removing outliers") 
        cols_to_normalize = ["actual_total_time",
                              "optimal_total_time",
                              "diff_actual_optimal_time",
                              "start_delay",
                              "end_delay",
                              "PTP"]

        for col in cols_to_normalize:
            col_zscore = col + '_zscore'
            df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)
        
        zcore_cols  =  [col+ '_zscore' for col in cols_to_normalize]
        filters = [(df[col]>-3) & (df[col]<3) for col in zcore_cols]

        df_filtered = df[np.logical_and.reduce(filters)]

        logger.info("Dropping not neccesary columns") 
        df_clean = df_filtered.drop(columns=["actual_start_time_picking","actual_end_time_picking","actual_total_time","diff_actual_optimal_time","PTP",*zcore_cols])

        logger.info("Saving list of stores")
        save_object(os.path.join(self.config.root_dir,"stores.pkl"),list(df_clean["store_id"].unique()))

        logger.info("Saving preprocessing data")
        df_clean.to_csv(self.config.clean_data_file, index=False, header=True)

In [5]:
try:
    config = ConfigurationManager()
    data_preprocessing_config = config.get_data_preprocessing_config()
    data_preprocessing = DataPreprocessing(config=data_preprocessing_config)
    data_preprocessing.start()
except Exception as e:
    raise e

[2024-04-20 14:53:10,603] 29 common - INFO - yaml file config\config.yml loaded successfully
[2024-04-20 14:53:10,604] 47 common - INFO - Created directory at: artifacts
[2024-04-20 14:53:10,605] 47 common - INFO - Created directory at: artifacts/data_preprocessing
[2024-04-20 14:53:10,606] 12 1439955191 - INFO - Read the dataset as dataframe
[2024-04-20 14:53:11,885] 15 1439955191 - INFO - Converting picking times to datetime DType
[2024-04-20 14:54:29,456] 24 1439955191 - INFO - Removing missing values
[2024-04-20 14:54:29,473] 27 1439955191 - INFO - Adding columns for analysis
[2024-04-20 14:54:29,526] 37 1439955191 - INFO - Removing outliers
[2024-04-20 14:54:29,623] 54 1439955191 - INFO - Dropping not neccesary columns
[2024-04-20 14:54:29,639] 57 1439955191 - INFO - Saving preprocessing data
[2024-04-20 14:54:35,184] 60 1439955191 - INFO - Saving list of stores
